In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Dropout
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error
# from sklearn import metrics

In [2]:
from tqdm.notebook import tqdm
import warnings

In [11]:
device_full_files = pd.DataFrame()


with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
    for year in tqdm([2017, 2018, 2019, 2020, 2022, 2023]):
        device_file = pd.read_csv('https://www.accessdata.fda.gov/MAUDE/ftparea/device%d.zip' % year, sep='|', quoting=3, encoding = "ISO-8859-1", on_bad_lines='skip')
        device_file_filtered = device_file[device_file['GENERIC_NAME'].str.contains("MESH, SURGICAL, SYNTHETIC, UROGYNECOLOGIC, FOR STRESS URINARY INCONTINENCE, RETR").fillna(False)]
        device_file_filtered.to_csv('./data/device_filtered%d.zip' % year, index = False)
        # device_full_files = device_full_files.append(device_file_filtered)
        device_full_files = pd.concat([device_full_files, pd.DataFrame(device_file_filtered)], ignore_index=True)

  0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
patient = pd.read_csv('./data/device2019.zip', sep='|', quoting=3, encoding = "ISO-8859-1", on_bad_lines='skip')

/var/folders/xt/7gvkdnw143dgfgsp73bsn11w0000gn/T/ipykernel_1090/4273465210.py:1: DtypeWarning: Columns (0,1,2,3,4,22) have mixed types. Specify dtype option on import or set low_memory=False.
  patient = pd.read_csv('./data/device2019.zip', sep='|', quoting=3, encoding = "ISO-8859-1", on_bad_lines='skip')


In [6]:
patient.head()

,MDR_REPORT_KEY,DEVICE_EVENT_KEY,IMPLANT_FLAG,DATE_REMOVED_FLAG,DEVICE_SEQUENCE_NO,DATE_RECEIVED,BRAND_NAME,GENERIC_NAME,MANUFACTURER_D_NAME,MANUFACTURER_D_ADDRESS_1,...,LOT_NUMBER,OTHER_ID_NUMBER,DEVICE_AVAILABILITY,DATE_RETURNED_TO_MANUFACTURER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,COMBINATION_PRODUCT_FLAG,UDI-DI,UDI-PUBLIC
0,7491296,NaN,NaN,NaN,1,2019/03/06,PARADIGM REAL-TIME INSULIN INFUSION PUMP MMT-7...,"PUMP, INFUSION, INSULIN, TO BE USED WITH INVAS...",MEDTRONIC PUERTO RICO OPERATIONS CO.,CEIBA NORTE IND. PARK #50 ROAD,...,A4722NASJ,NaN,R,2018/04/11,OYC,DA,R,N,00613994373083,(01)00613994373083
1,7811695,NaN,NaN,NaN,1,2019/03/08,"MONOPOLAR ACTIVE CORD, BOVIE/VALLEYLAB GENERATORS",MONOPOLAR ACTIVE CORD,"GYRUS ACMI, INC",136 TURNPIKE ROAD,...,1K,NaN,R,2019/02/07,FAS,DA,Y,N,NaN,NaN
2,7987287,NaN,NaN,NaN,1,2019/07/18,PARADIGM REAL-TIME REVEL INSULIN INFUSION PUMP...,"PUMP, INFUSION, INSULIN, TO BE USED WITH INVAS...",MEDTRONIC PUERTO RICO OPERATIONS CO.,CEIBA NORTE IND. PARK #50 ROAD,...,A5723LNALJ,NaN,R,2019/07/10,OYC,DA,R,N,00643169513891,(01)00643169513891
3,8033177,NaN,NaN,NaN,1,2019/02/06,PROMAX¿ SERIES BARD® C-MAX¿ CUTTING LOOP,CUTTING LOOP,"C.R. BARD, INC. (COVINGTON) -1018233",8195 INDUSTRIAL BLVD,...,UNK,NaN,N,NaN,FAS,DA,R,N,00801741018572,(01)00801741018572
4,8155636,NaN,NaN,NaN,1,2019/01/03,ENDO ANCHOR SYSTEM - HELI-FX AAA,ENDOVASCULAR SUTURING SYSTEM,MEDTRONIC IRELAND,PARKMORE BUSINESS PARK WEST,...,NaN,NaN,N,NaN,OTD,DA,*,N,NaN,NaN


In [7]:
patient_filtered = patient[patient['GENERIC_NAME'].str.contains("MESH, SURGICAL, SYNTHETIC, UROGYNECOLOGIC, FOR STRESS URINARY INCONTINENCE, RETR").fillna(False)]

In [10]:
patient_filtered.shape

(238, 31)

In [12]:
device_full_files.shape

(2034, 31)